# Prerequisite

Following packages must be installed to run codes below

```bash
$ pip3 install openpyxl 
```


In [2]:
%load_ext autoreload
%autoreload 2

# Add project path

In [2]:
import os
import sys
from pathlib import Path

PROJECT_PATH = Path().resolve().parent.parent
SRC_PATH = PROJECT_PATH / 'src'

sys.path.append(str(SRC_PATH))

# Load data

In [3]:
from project_paths import DATA_PATH
from data_loader import DataLoader
import pandas as pd
pd.set_option('display.float_format', lambda x: '%.3f' % x)
data_loader = DataLoader(data_path=DATA_PATH)

In [4]:
meta_data = data_loader.load_meta_data()
description_data_code = data_loader.load_description_data(sheet_name='코드집')
description_data_financial_code = data_loader.load_description_data(sheet_name='재무계정')
ksic_data = data_loader.load_ksic_data()
finance_data = data_loader.load_finance_data()

In [5]:
finance_data.iloc[:2].T

,0,1
사업자번호,1018100340.000,1018100340.000
결산년월,20181231.000,20191231.000
유동자산,2667731.000,2168837.000
매출채권,127320.000,94085.000
비유동자산,32346444.000,34151633.000
유형자산,1179096.000,1209369.000
자산총계,35014174.000,36320470.000
유동부채,422872.000,10553878.000
비유동부채,16176067.000,5240445.000
부 채 총 계,16598939.000,15794323.000


In [6]:
import pandas as pd
from typing import List
import numpy as np

def as_int(x) : 
    try : 
        x = int(x)
    except :
        x = np.nan
    return x

company_data = (
    pd.concat(
        [
            (
                data_loader.load_company_data(closed=False)
                .assign(Source='Active')
            ),
            (
                data_loader.load_company_data(closed=True)
                .assign(Source='Closed')
            )
        ],
        axis=0
    )
)

def get_korean_column_name_dict(data:pd.DataFrame) -> List : 
    column_intersection = list(
        set(
            meta_data['영문칼럼명']
            .tolist()
        )
        .intersection(
            set(
                data
                .columns
                .tolist()
            )
        )
    )

    column_mapping = (
        meta_data
        .loc[lambda x : x['영문칼럼명'].isin(column_intersection), ['영문칼럼명', '한글칼럼명']]
        .set_index('영문칼럼명')
        .to_dict()
        ['한글칼럼명']
    )
    
    return column_mapping

company_data = (
    company_data
    .merge(
        (
            ksic_data
            .assign(세세분류_코드=lambda x : x['세세분류_코드'].astype(float))
        ),
        left_on='IND_CD1',
        right_on='세세분류_코드',
        how='left'
    )
    .assign(Closed=lambda x : x['CLSBZ_GB'].apply(lambda y : True if y==3 else False))
    #.rename(columns=column_mapping)
)

In [7]:
import plotly.express as px
import plotly.io as pio
pio.renderers.default = 'iframe'

In [8]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['대분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['유동부채'].T
px.line(data_toplot, markers=True, title='대분류별 폐업기업 유동부채 총계')

In [9]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['대분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['부  채  총  계'].T
px.line(data_toplot, markers=True, title='대분류별 폐업기업 총부채총계')

In [10]:
data_toplot = (
    company_data
    #.loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='right'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['대분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['유동부채'].T
px.line(data_toplot, markers=True, title='대분류별 전체기업 유동부채 총계')

In [11]:
data_toplot = (
    company_data
    #.loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='right'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['대분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['부  채  총  계'].T
px.line(data_toplot, markers=True, title='대분류별 전체기업 총부채 총계')

In [12]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['중분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['유동부채'].T
px.line(data_toplot, markers=True, title='중분류별 폐업기업 유동부채 총계')

In [13]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['중분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['부  채  총  계'].T
px.line(data_toplot, markers=True, title='중분류별 폐업기업 총부채총계')

In [14]:
data_toplot = (
    company_data
    #.loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='right'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['중분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['유동부채'].T
px.line(data_toplot, markers=True, title='중분류별 전체기업 유동부채 총계')

In [15]:
data_toplot = (
    company_data
    #.loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='right'
    )
    .loc[lambda x : x['결산년월'].notnull()]
    .assign(결산년월 = lambda x : pd.to_datetime(x['결산년월'].astype(int), format='%Y%m%d'))
    .assign(결산년=lambda x : x['결산년월'].dt.year)
    .groupby(['중분류', '결산년'])
    [['유동부채', '부  채  총  계']].sum()
    .unstack()
    #.sort_values(by='유동부채', ascending=False)
)['부  채  총  계'].T
px.line(data_toplot, markers=True, title='중분류별 전체기업 총부채 총계')

In [16]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .groupby(['대분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)
fig = px.bar(data_toplot, 
             orientation='h', 
             title='대분류별 폐업시점 부채총계')
fig.show()

In [1]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .groupby(['대분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
    .iloc[:5]
)
fig = px.bar(data_toplot, 
             orientation='h', 
             title='대분류별 폐업시점 부채총계')
fig.show()

NameError: name 'company_data' is not defined

In [17]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
    .tail()
)
fig = px.bar(data_toplot, 
             orientation='h', 
             title='중분류별 폐업시점 부채총계')
fig.show()

In [18]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .groupby(['세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
    .tail()
)
fig = px.bar(data_toplot, 
             orientation='h', 
             title='세분류별 폐업시점 부채총계')
fig.show()

In [19]:
data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .groupby(['세세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
    .tail()
)
fig = px.bar(data_toplot, 
             orientation='h', 
             title='세세분류별 폐업시점 부채총계')
fig.show()

In [20]:
level_1_candidates = [
    '제조업(10~34)',
    '부동산업(68)',
    '도매 및 소매업(45~47)',
    '숙박 및 음식점업(55~56)',
    '건설업(41~42)'
]

In [21]:
level_1 = level_1_candidates[0]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'중분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [22]:
level_1 = level_1_candidates[1]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'중분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [23]:
level_1 = level_1_candidates[1]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'세분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [24]:
level_1 = level_1_candidates[2]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'중분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [25]:
level_1 = level_1_candidates[2]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'세분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=1000,)
fig.show()

In [26]:
level_1 = level_1_candidates[3]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'중분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [27]:
level_1 = level_1_candidates[3]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'세분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [28]:
level_1 = level_1_candidates[4]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['중분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'중분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()

In [29]:
level_1 = level_1_candidates[4]

data_toplot = (
    company_data
    .loc[lambda x : x['Closed']==True]
    .merge(
        (
            finance_data
            .groupby(['사업자번호'])
            [finance_data.columns]
            .last()
            .reset_index(drop=True)
        ),
        left_on='BIZ_NO',
        right_on='사업자번호',
        how='left'
    )
    .loc[lambda x : x['대분류']==level_1]
    .groupby(['세분류'])
    [['유동부채', '비유동부채', '부  채  총  계']].sum()
    .sort_values(by='부  채  총  계', ascending=True)
    .drop('부  채  총  계', axis=1)
)

fig = px.bar(data_toplot, 
             orientation='h', 
             title=f'세분류별 폐업시점 부채총계 : {level_1}')
fig.update_layout(
    autosize=False,
    width=800,
    height=800,)
fig.show()